In [2]:
import h5py
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

def run_peak_fit_model_on_insertion(self):
    # get relevant data
    waves = np.array(config.current_wavelength_vector)
    absorbance_df = self.create_absorbance_df(1)
    # run the model
    modelparams, modelcovar, modeled_spectra, residual_spectra = model_fit(absorbance_df, waves)
    # create a dataframe for the modeled spectra
    modeled_df = pd.DataFrame(data=modeled_spectra)
    # determine standard error for the f parameter and add it to the dataframe
    sd_err = extract_f_sd_err(modelcovar)
    modeled_df['sd_err'] = sd_err
    # determine root mean square error for the model fit and add it to the dataframe
    squared_errors = np.square(residual_spectra)
    mse = np.mean(squared_errors, axis=1)
    rmse = np.sqrt(mse)
    modeled_df['rmse'] = rmse
    # add the model parameters to the dataframe
    modeled_df[['c0', 'c1', 'c2', 'c3', 'w1', 'w2', 'f1']] = modelparams
    return modeled_df

def create_absorbance_df(self, spectrometer):
    if spectrometer == 1:
        absorbances = np.asarray(self.insertion.absorbances)
        timestamps = timestamp_list_to_pandas_timestamps(self.insertion.spectrum_timestamps)

def timestamp_list_to_pandas_timestamps(timestamp_list):
    series = pd.Series(timestamp_list).astype('<M8[us]')
    return series

def spec_fit(x, c0, c1, c2, c3, w1, w2, f1):
    '''
    A function for fitting peaks to spectra.
    This fit function was copy-pasted in as text from David's fit function on 5/11/21.
    There are two water peaks (free water and bound water) modeled as gaussian peaks w1 and w2.
    There is the f1 fatty acid peak.
    There is a three-term polynomial function.

    :param x: input data
    :param c0: polynomial term 1
    :param c1: polynomial term 2
    :param c2: polynomial term 3
    :param c3: polynomial term 4
    :param w1: water peak term 1
    :param w2: water peak term 2
    :param f1: fatty acid peak
    :return:
    '''

    f_of_x = c0+c1*(x-1700)+c2*(x-1700)**2+c3*(x-1700)**3 + \
             w1*(0.0751747*np.exp(-(x-1903.82)**2/26.4725**2)+0.225213*np.exp(-(x-1942.21)**2/48.8781**2) +
                 0.005*np.exp(-(x-1779.71)**2/32.1869**2))/7.715 + \
             w2*(0.0280945*np.exp(-(x-1913.6)**2/25.0449**2)+0.103527*np.exp(-(x-1949.5)**2/52.2024**2))/3.07 + \
             f1*(0.31*np.exp(-(x-(1730-24))**2/17**2)+np.exp(-(x-1730)**2/17**2)+0.39 *
                 np.exp(-(x-(1730+31))**2/17**2))/25.484
    return f_of_x

def model_fit(df, wave_array):
    '''
    Performs spec_fit function on the input spectra.

    Input parameters:
    df is dataframe containing only the spectra, without extraneous columns
    wave_array is np.array containing the wavelengths of spectra

    Return values:
    modelparams is np.array of parameters output from curve_fit
    modelcovar is np.array of covariance array output from curve_fit
    modeled_spectra is np.array of the modeled spectra using the curve_fit parameters
    residual_spectra is np.array of the original spectrum - the modeled spectrum

    '''
    number = df.shape[0]
    modelparams = np.empty((number, 7), dtype=float)
    modelcovar = np.empty((number, 7, 7), dtype=float)
    modeled_spectra = np.empty((number, wave_array.shape[0]), dtype=float)
    residual_spectra = np.empty((number, wave_array.shape[0]), dtype=float)
    for i in range(number):
        row = df.iloc[i, :]
        modeled, pcov = curve_fit(spec_fit, wave_array, row)
        modelparams[i, :] = modeled
        modelcovar[i, ::] = pcov
        modeled_spectra[i, :] = spec_fit(wave_array, *modeled)
        residual_spectra[i, :] = row - modeled_spectra[i, :]
    return modelparams, modelcovar, modeled_spectra, residual_spectra


def extract_farray(modelparams):
    '''
    Convenience function to create an np.array containing only the f1 values

    Input:  the modelparams np.array created by model_fit().
    '''
    return modelparams[:, 6]


def create_modelparams_df(modelparams):
    columns = ['c0', 'c1', 'c2', 'c3', 'w1', 'w2', 'f1']
    modelparams_df = pd.DataFrame(data=modelparams, columns=columns)
    return modelparams_df


def extract_f_sd_err(modelcovar):
    '''
    Convenience function to create an np.array containing the standard deviation
       of the covariance matrix for the f1 term.

    Input:  the modelcovar np.array created by model_fit().

    '''
    f_sd_err = np.empty(modelcovar.shape[0])
    for i in range(modelcovar.shape[0]):
        f_sd_err[i] = np.diag(modelcovar[i, :, :])[6]
    return f_sd_err


def high_f1_numbers(farray, threshold):
    '''
    Returns np.array of the index positions of spectra with f1 values greater than the
       threshold.  Use this with iloc to find the appropriate rows in the dataframe.

    Input:  np.array output from extract_farray() and desired threshold value.
    '''
    return np.where(farray > threshold)[0]

In [3]:
file = "data/blah_beta.h5"

In [9]:
with h5py.File(file, 'r') as h5_file:
    print(h5_file['session048/cal001/ins001/derived/modeled']
    [:].shape)

(19, 410)


In [5]:
with h5py.File(file, 'r') as h5_file:
    print(h5_file['/'].keys())

<KeysViewHDF5 ['session001', 'session002', 'session003', 'session004', 'session005', 'session006', 'session007', 'session008', 'session009', 'session010', 'session011', 'session012', 'session013', 'session014', 'session015', 'session016', 'session017', 'session018', 'session019', 'session020', 'session021', 'session022', 'session023', 'session024', 'session025', 'session026', 'session027', 'session028', 'session029', 'session030', 'session031', 'session032', 'session033', 'session034', 'session035', 'session036', 'session037', 'session038', 'session039', 'session040', 'session041', 'session042', 'session043', 'session044', 'session045', 'session046', 'session047', 'session048']>


In [10]:
with h5py.File(file, 'r') as h5_file:
    print(h5_file['session048/cal001/ins001/derived/modeled']
    [:, -1])

[ 0.86693888 -0.00611296 -0.00562552 -0.00874377 -0.01528805 -0.0156569
 -0.01972628 -0.0303306  -0.04017102 -0.03718631 -0.0308056  -0.04583852
 -0.04126997 -0.03821567 -0.05596053 -0.04945316 -0.04045875 -0.04692674
 -0.05424555]


In [15]:
columns = list(range(0, 401, 1))
modeled_columns = ['sd_err', 'rmse', 'c0', 'c1', 'c2', 'c3', 'w1', 'w2', 'f1']
columns = columns + modeled_columns

In [18]:
with h5py.File(file, 'r') as h5_file:
    data = h5_file['session048/cal001/ins001/derived/modeled'][:]

In [19]:
df = pd.DataFrame(data=data, columns=columns)

In [20]:
df

,0,1,2,3,4,5,6,7,8,9,...,400,sd_err,rmse,c0,c1,c2,c3,w1,w2,f1
0,0.412516,0.412053,0.411603,0.411166,0.410742,0.410331,0.409933,0.409547,0.409174,0.408813,...,0.421754,0.010423,0.016542,0.440431,0.000507,-8.743775e-08,-1.486287e-08,-1.743454,4.094606,0.866939
1,0.369179,0.371311,0.373425,0.375522,0.377601,0.379662,0.381706,0.383733,0.385742,0.387733,...,0.651657,0.017290,0.021305,0.527290,0.000219,-3.884420e-06,1.120707e-08,6.037985,-1.074594,-0.006113
2,0.514812,0.514518,0.514226,0.513936,0.513648,0.513363,0.513079,0.512798,0.512519,0.512242,...,0.660836,0.000032,0.000921,0.494537,0.000024,1.061314e-06,-1.844708e-11,4.250832,-0.827179,-0.005626
3,0.507566,0.507225,0.506887,0.506551,0.506217,0.505885,0.505556,0.505229,0.504904,0.504582,...,0.656709,0.000036,0.000972,0.482343,0.000007,1.179224e-06,7.046450e-11,4.621883,-1.031031,-0.008744
4,0.453858,0.453575,0.453295,0.453016,0.452739,0.452463,0.452190,0.451918,0.451648,0.451379,...,0.582922,0.000019,0.000710,0.431641,-0.000010,9.376713e-07,1.203388e-10,3.729818,-0.417619,-0.015288
5,0.494295,0.494016,0.493738,0.493462,0.493188,0.492916,0.492646,0.492378,0.492112,0.491848,...,0.627160,0.000030,0.000890,0.473775,0.000007,9.572353e-07,-3.810306e-12,3.702646,-0.436319,-0.015657
6,0.507391,0.507111,0.506834,0.506558,0.506284,0.506013,0.505743,0.505475,0.505209,0.504946,...,0.642110,0.000023,0.000770,0.487396,0.000012,9.498482e-07,-1.094611e-10,3.694965,-0.365349,-0.019726
7,0.460152,0.459912,0.459674,0.459437,0.459202,0.458968,0.458736,0.458506,0.458277,0.458050,...,0.572131,0.000014,0.000608,0.442237,0.000003,8.243434e-07,4.931124e-11,3.126869,-0.390981,-0.030331
8,0.484669,0.484419,0.484172,0.483925,0.483681,0.483438,0.483196,0.482957,0.482719,0.482483,...,0.608622,0.000015,0.000631,0.466210,0.000007,8.810708e-07,1.065544e-10,3.431258,-0.437679,-0.040171
9,0.470571,0.470340,0.470111,0.469883,0.469657,0.469432,0.469209,0.468987,0.468767,0.468548,...,0.591647,0.000014,0.000602,0.453134,0.000001,8.014427e-07,1.109629e-10,3.133936,-0.124245,-0.037186


In [23]:
with h5py.File(file, 'r') as h5_file:
    print(h5_file['session048/cal001/ins001/derived/absorbance_depth'][:])

[[4.59480934e-01 4.56682426e-01 4.54583480e-01 ... 4.57605516e-01
  9.65800000e+04 2.58638020e+00]
 [3.87091487e-01 3.85584130e-01 3.83795232e-01 ... 6.64153602e-01
  1.93914000e+05 5.09295273e+00]
 [5.15795914e-01 5.15263627e-01 5.14979515e-01 ... 6.63474248e-01
  1.15113000e+05 7.59068756e+00]
 ...
 [4.98512942e-01 4.98043627e-01 4.97903911e-01 ... 6.38535520e-01
  9.76958000e+05 3.88172455e+01]
 [5.12645402e-01 5.12629890e-01 5.12051108e-01 ... 6.46022599e-01
  9.25801000e+05 4.00892242e+01]
 [5.16828221e-01 5.16560717e-01 5.16128984e-01 ... 6.57781292e-01
  4.19530000e+05 4.09530762e+01]]


In [26]:
with h5py.File(file, 'r') as h5_file:
    depths = h5_file['session048/cal001/ins001/derived/absorbance_depth'][:, -1]

In [27]:
depths

array([ 2.5863802 ,  5.09295273,  7.59068756, 10.09070816, 12.59062653,
       15.09039307, 17.59031219, 20.09043427, 22.59035339, 25.09037476,
       27.59013977, 30.09000854, 32.58972473, 33.51352234, 35.08948975,
       37.58930969, 38.81724548, 40.08922424, 40.95307617])

In [28]:
df['depth'] = depths

In [29]:
df

,0,1,2,3,4,5,6,7,8,9,...,sd_err,rmse,c0,c1,c2,c3,w1,w2,f1,depth
0,0.412516,0.412053,0.411603,0.411166,0.410742,0.410331,0.409933,0.409547,0.409174,0.408813,...,0.010423,0.016542,0.440431,0.000507,-8.743775e-08,-1.486287e-08,-1.743454,4.094606,0.866939,2.586380
1,0.369179,0.371311,0.373425,0.375522,0.377601,0.379662,0.381706,0.383733,0.385742,0.387733,...,0.017290,0.021305,0.527290,0.000219,-3.884420e-06,1.120707e-08,6.037985,-1.074594,-0.006113,5.092953
2,0.514812,0.514518,0.514226,0.513936,0.513648,0.513363,0.513079,0.512798,0.512519,0.512242,...,0.000032,0.000921,0.494537,0.000024,1.061314e-06,-1.844708e-11,4.250832,-0.827179,-0.005626,7.590688
3,0.507566,0.507225,0.506887,0.506551,0.506217,0.505885,0.505556,0.505229,0.504904,0.504582,...,0.000036,0.000972,0.482343,0.000007,1.179224e-06,7.046450e-11,4.621883,-1.031031,-0.008744,10.090708
4,0.453858,0.453575,0.453295,0.453016,0.452739,0.452463,0.452190,0.451918,0.451648,0.451379,...,0.000019,0.000710,0.431641,-0.000010,9.376713e-07,1.203388e-10,3.729818,-0.417619,-0.015288,12.590627
5,0.494295,0.494016,0.493738,0.493462,0.493188,0.492916,0.492646,0.492378,0.492112,0.491848,...,0.000030,0.000890,0.473775,0.000007,9.572353e-07,-3.810306e-12,3.702646,-0.436319,-0.015657,15.090393
6,0.507391,0.507111,0.506834,0.506558,0.506284,0.506013,0.505743,0.505475,0.505209,0.504946,...,0.000023,0.000770,0.487396,0.000012,9.498482e-07,-1.094611e-10,3.694965,-0.365349,-0.019726,17.590312
7,0.460152,0.459912,0.459674,0.459437,0.459202,0.458968,0.458736,0.458506,0.458277,0.458050,...,0.000014,0.000608,0.442237,0.000003,8.243434e-07,4.931124e-11,3.126869,-0.390981,-0.030331,20.090434
8,0.484669,0.484419,0.484172,0.483925,0.483681,0.483438,0.483196,0.482957,0.482719,0.482483,...,0.000015,0.000631,0.466210,0.000007,8.810708e-07,1.065544e-10,3.431258,-0.437679,-0.040171,22.590353
9,0.470571,0.470340,0.470111,0.469883,0.469657,0.469432,0.469209,0.468987,0.468767,0.468548,...,0.000014,0.000602,0.453134,0.000001,8.014427e-07,1.109629e-10,3.133936,-0.124245,-0.037186,25.090375


In [31]:
with h5py.File(file, 'r') as h5_file:
    print(h5_file['session048/cal001/ins004/derived/modeled'][:, -1])

[-0.01765918 -0.01734964  0.14890162  0.07764754 -0.0119282  -0.03711295
 -0.03495717 -0.03282838  0.80315189 -0.04365915 -0.03523946 -0.04403358]


In [32]:
with h5py.File(file, 'r') as h5_file:
    print(h5_file['session048/cal001/ins004/spectrometer1'].keys())

<KeysViewHDF5 ['derived', 'spectra', 'spectrum_depths', 'spectrum_forces', 'timestamps']>
